### Sorting, abstract class

In [1]:
import numpy as np
import functools
import copy
import operator


class Sorting(object):
    """排序算法抽象类
    
    - 通用功能：交换和移动
        + swap()
        + move()
    - 数据生成： random, almost sorted, reversed, few unique
    - 性能分析： time: #swap, #move; space: 
    """
    
    def __init__(self):
        
        # run & performance info
        self.origin_L = None
        self.L = None
        self.N = None
        
        self.count_call_dict = {}
        
        self.alg_cmp = self.count_call(self.alg_cmp)
        self.swap = self.count_call(self.swap)
        self.copy_to = self.count_call(self.copy_to)
    
    @staticmethod
    def generate_L(N=30, few_unique_nvalue=5, seed=123456):
        np.random.seed(seed)
        return {
            'L_random': list(np.random.choice(range(N), size=N, replace=False)),
            'L_sorted': list(range(N)),
            'L_reversed': list(range(N, 0, -1)),
            'L_few_unique': list(np.random.choice(range(few_unique_nvalue), size=N, replace=True)),
        }
    
    # ------------ sort -----------------------------
    def sort(self, L_):
        L = copy.deepcopy(L_)
        if True:
            self.origin_L = copy.deepcopy(L)
            self.L = L
            self.N = len(L)
            
        self._sort(self.L)
        
        if True:
            Sorting.check_sorted(self.L)
            self.perfromance_summary()
        
    def _sort(self, L):
        raise NotImplementedError
        
    # ------------ utils && performance analysis -----------------------------    
    def count_call(self, func):
        self.count_call_dict[func.__name__] = 0
        @functools.wraps(func)
        def wrapper(*args, **kw):
            self.count_call_dict[func.__name__] += 1
            return func(*args, **kw)
        return wrapper
    
    def alg_cmp(self, a, op, b):
        return op(a, b)
    
    def swap(self, L, i, j):
        L[i], L[j] = L[j], L[i]
        
    def copy_to(self, L, i, value):
        # L[i] = value   # value should be object that pass by value
        L[i] = copy.deepcopy(value)   # just in case of
            
    def perfromance_summary(self):
        print(self.origin_L)
        print('inversion number:', Sorting.get_inversion_number(self.origin_L))
        print(self.L, Sorting.check_sorted(self.L))
        print('------------------------')
        print('N =', self.N)
        print('========= Theretical Analysis =========')
        for (txt, f) in self.theretical_complexity():
            print(txt, '-->', str(f(self.N)))
        print('------------------------')
        print('time complexity: ', self.count_call_dict)
        print('space complexity:', )
        
    def theretical_complexity(self):
        raise NotImplementedError
        
    @staticmethod
    def get_inversion_number(L):
        n = len(L)
        return sum(L[i] > L[j] for i in range(n) for j in range(i+1, n))
    
    @staticmethod
    def check_sorted(L):
        return all(L[i] <= L[i+1] for i in range(len(L) - 1))

In [2]:
data = Sorting.generate_L(N=20)
L1 = data['L_random']
L2 = data['L_few_unique']

### Bubble Sort

In [47]:
class BubbleSort(Sorting):
    """冒泡排序"""
    
    def _sort(self, L):
        n = len(L)
        for i in range(n):
            for j in range(n-1, i, -1):
                if self.alg_cmp(L[j-1], operator.__gt__, L[j]):
                    self.swap(L, j-1, j)
                    
    def theretical_complexity(self):
        return [
            ('#cmp: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  bad-case reversed: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  good-case sorted: 0', lambda N: 0),
            ('#space: 0', lambda N: 0)
        ]

In [48]:
m_bb = BubbleSort()
m_bb.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case reversed: N(N-1)/2 --> 190.0
#swap  good-case sorted: 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 101, 'copy_to': 0}
space complexity:


In [49]:
m_bb2 = BubbleSort()
m_bb2.sort(L2)

[4, 4, 4, 4, 3, 1, 4, 3, 3, 0, 4, 2, 2, 0, 3, 4, 2, 1, 2, 4]
inversion number: 100
[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case reversed: N(N-1)/2 --> 190.0
#swap  good-case sorted: 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 100, 'copy_to': 0}
space complexity:


### Selection Sort

In [50]:
class SelectionSort(Sorting):
    """选择排序"""
    
    def _sort(self, L):
        n = len(L)
        for i in range(n):
            min_idx = i
            for j in range(i+1, n):
                if self.alg_cmp(L[j], operator.__lt__, L[min_idx]):
                    min_idx = j
            if min_idx > i:
                self.swap(L, i, min_idx)  # if min_idx > i, 这个比较算不算在时间复杂度内？
                    
    def theretical_complexity(self):
        return [
            ('#cmp: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  bad-case(most case, swap for all i): N', lambda N: N),
            ('#swap  good-case(sorted): 0', lambda N: 0),
            ('#space: 0', lambda N: 0)
        ]

In [51]:
m_selection = SelectionSort()
m_selection.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case(most case, swap for all i): N --> 20
#swap  good-case(sorted): 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 17, 'copy_to': 0}
space complexity:


In [52]:
m_selection2 = SelectionSort()
m_selection2.sort(L2)

[4, 4, 4, 4, 3, 1, 4, 3, 3, 0, 4, 2, 2, 0, 3, 4, 2, 1, 2, 4]
inversion number: 100
[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case(most case, swap for all i): N --> 20
#swap  good-case(sorted): 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 11, 'copy_to': 0}
space complexity:


In [53]:
m_selection3 = SelectionSort()
m_selection3.sort([0, 0, 0, 1, 1, 1])

[0, 0, 0, 1, 1, 1]
inversion number: 0
[0, 0, 0, 1, 1, 1] True
------------------------
N = 6
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 15.0
#swap  bad-case(most case, swap for all i): N --> 6
#swap  good-case(sorted): 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 15, 'swap': 0, 'copy_to': 0}
space complexity:


### Insertion Sort

In [69]:
class InsertionSort(Sorting):
    """插入排序"""
    
    def _sort(self, L):
        n = len(L)
        L.append(None)
        TMP = n
        
        # [  0, x, x, x, x, x, x, x,   i-1,   i,  ]
        # [j=0 <------------j------- j=i-1,   i,  ]
        # [                 k >>>>>>>>>>>>>>>>|,  ]
        for i in range(n):
            for j in reversed(range(i)):
                if self.alg_cmp(L[i], operator.__ge__, L[j]):  # L[i] 放到 j 后面
                    self.copy_to(L, TMP, L[i])
                    # tmp_storage = copy.deepcopy(L[i])  # todo: copy_to
                    for k in range(i, j+1, -1):
                        self.copy_to(L, k, L[k-1])
                        #L[k] = copy.deepcopy(L[k-1])
                    self.copy_to(L, j+1, L[TMP])
                    #L[j+1] = copy.deepcopy(tmp_storage)
                    break
                    
            # here j == 0, if not break
            if L[i] < L[0]:
                self.copy_to(L, TMP, L[i])
                # tmp_storage = copy.deepcopy(L[i])
                for k in range(i, 0, -1):
                    self.copy_to(L, k, L[k-1])
                    #L[k] = copy.deepcopy(L[k-1])
                self.copy_to(L, 0, L[TMP])
                #L[0] = copy.deepcopy(tmp_storage)
        del L[TMP]
                    
    def theretical_complexity(self):
        return [
            ('#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#cmp good-case(sorted): N-1', lambda N: N-1),
            ('#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2', lambda N: (N+4) * (N-1) / 2),
            ('#copy  good-case(sorted): 2(N-1)', lambda N: 2 * (N-1)),
            ('#space: 1', lambda N: 1)
        ]

In [70]:
m_insert = InsertionSort()
m_insert.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2 --> 190.0
#cmp good-case(sorted): N-1 --> 19
#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2 --> 228.0
#copy  good-case(sorted): 2(N-1) --> 38
#space: 1 --> 1
------------------------
time complexity:  {'alg_cmp': 114, 'swap': 0, 'copy_to': 139}
space complexity:


In [71]:
m_insert2 = InsertionSort()
m_insert2.sort(L2)

[4, 4, 4, 4, 3, 1, 4, 3, 3, 0, 4, 2, 2, 0, 3, 4, 2, 1, 2, 4]
inversion number: 100
[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2 --> 190.0
#cmp good-case(sorted): N-1 --> 19
#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2 --> 228.0
#copy  good-case(sorted): 2(N-1) --> 38
#space: 1 --> 1
------------------------
time complexity:  {'alg_cmp': 116, 'swap': 0, 'copy_to': 138}
space complexity:


In [74]:
m_insert3 = InsertionSort()
m_insert3.sort([0, 1, 2, 3, 4, 5, 1])

[0, 1, 2, 3, 4, 5, 1]
inversion number: 4
[0, 1, 1, 2, 3, 4, 5] True
------------------------
N = 7
========= Theretical Analysis =========
#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2 --> 21.0
#cmp good-case(sorted): N-1 --> 6
#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2 --> 33.0
#copy  good-case(sorted): 2(N-1) --> 12
#space: 1 --> 1
------------------------
time complexity:  {'alg_cmp': 10, 'swap': 0, 'copy_to': 16}
space complexity:


## Shell Sort: to FIX

In [193]:
class ShellSort(Sorting):
    """希尔排序"""
    
    def _sort(self, L):
        n = len(L)
        inc = n
        tmpL = [None]
        while inc > 1:
            inc = int(inc / 3 + 1)
            for i in range(inc):
                idxs = list(range(i, n, inc))
                print('---> ', idxs)
                # [    ==     ==     ==    ==      ==   ==   ]
                # [    ==      ==    j     last_j  ==   ==   ]
                #                                       tmpL ]
                
                self.copy_to(tmpL, 0, L[idxs[-1]])
                print('----> tmpL = ', tmpL[0])
                
                last_j = idxs[-1]
                for j in reversed(idxs[:-1]):
                    print(j, last_j)
                    if L[j] > tmpL[0]:
                        print('move>>>', j, ' to ', last_j)
                        self.copy_to(L, last_j, L[j])
                        last_j = j
                    else:
                        print('move [tmpL] >>>>>> ', tmpL[0], 'to >>>>>', last_j)
                        self.copy_to(L, last_j, tmpL[0])
                        tmpL[0] = None
                        print('L is', L)
                        print('====== break ========')
                        last_j = j
                        break
                if j == idxs[0] and tmpL[0]:
                    print('hit the beginning of j, j=', j)
                    self.copy_to(L, j, tmpL[0])
                    print('L is ', L)
                print('--------------------------------')
        print('finished')

In [194]:
bool(np.nan)

True

In [195]:
ShellSort().sort([0, 1, 2, 3, 4, 5, -9])

--->  [0, 3, 6]
----> tmpL =  -9
3 6
move>>> 3  to  6
0 3
move>>> 0  to  3
hit the beginning of j, j= 0
L is  [-9, 1, 2, 0, 4, 5, 3]
--------------------------------
--->  [1, 4]
----> tmpL =  4
1 4
move [tmpL] >>>>>>  4 to >>>>> 4
L is [-9, 1, 2, 0, 4, 5, 3]
====== break ========
--------------------------------
--->  [2, 5]
----> tmpL =  5
2 5
move [tmpL] >>>>>>  5 to >>>>> 5
L is [-9, 1, 2, 0, 4, 5, 3]
====== break ========
--------------------------------
--->  [0, 2, 4, 6]
----> tmpL =  3
4 6
move>>> 4  to  6
2 4
move [tmpL] >>>>>>  3 to >>>>> 4
L is [-9, 1, 2, 0, 3, 5, 4]
====== break ========
--------------------------------
--->  [1, 3, 5]
----> tmpL =  5
3 5
move [tmpL] >>>>>>  5 to >>>>> 5
L is [-9, 1, 2, 0, 3, 5, 4]
====== break ========
--------------------------------
--->  [0, 1, 2, 3, 4, 5, 6]
----> tmpL =  4
5 6
move>>> 5  to  6
4 5
move [tmpL] >>>>>>  4 to >>>>> 5
L is [-9, 1, 2, 0, 3, 4, 5]
====== break ========
--------------------------------
finished
[0, 1, 2, 3, 4

NotImplementedError: 

## Merge Sort

In [20]:
class MergeSort(Sorting):
    """归并排序"""
    
    def _sort(self, L):
        # import pdb; pdb.set_trace()
        self._merge_sort(L, 0, len(L))
        
    def _merge_sort(self, L, start, end):
        n = len(L[start:end])
        
        if n == 0 or n == 1:
            return
        
        sep = start + int(n / 2)
        
        self._merge_sort(L, start, sep)
        self._merge_sort(L, sep, end)
        
        # [   start      SEP    end    ]
        # [     i >>>>>>>|j >>>>> |    ]
        
        i, j = start, sep
        new_L = []
        
        while True:
            if self.alg_cmp(L[i], operator.__le__, L[j]):
                new_L.append(L[i])
                i += 1
                if i >= sep:
                    new_L.extend(L[j:])
                    break
            else:
                new_L.append(L[j])
                j += 1
                if j >= end:
                    new_L.extend(L[i:sep])
                    break
                    
        for idx, k in zip(range(start, end), range(n)):
            L[idx] = new_L[k]
            
    def theretical_complexity(self):
        return [
            ('#cmp bad-case(reversed): nLog2(n)', lambda n: n * np.log2(n)),
            ('#cmp good-case(sorted): 1/2 * nLog2(n)', lambda n: 0.5 * n * np.log2(n)),
            ('#copy  [fix]: nLog2(n)', lambda n: n * np.log2(n)),
            ('#space: nLog2(n)', lambda n: n * np.log2(n))
        ]              

In [21]:
ms = MergeSort()
ms.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp bad-case(reversed): nLog2(n) --> 86.43856189774725
#cmp good-case(sorted): 1/2 * nLog2(n) --> 43.219280948873624
#copy  [fix]: nLog2(n) --> 86.43856189774725
#space: nLog2(n) --> 86.43856189774725
------------------------
time complexity:  {'alg_cmp': 63, 'swap': 0, 'copy_to': 0}
space complexity:


## Quick Sort: TOFIX

In [53]:
class QuickSort(Sorting):
    """快速排序"""
    
    def _sort(self, L):
        n = len(L)
        self.__quick_sort(L, 0, n)
        
    def __quick_sort(self, L, low, high):
        
        if low + 1 >= high:
            return
        
        pivoit = self.__partition(L, low, high)
        self.__quick_sort(L, low, pivoit)   # [low, pivoit)
        self.__quick_sort(L, pivoit, high)  # [pivoit, high)
        
    def __partition(self, L, low, high):
        
        print('==========================>>> partition')
        pivoit_value = np.mean(L[low:high])  # hack
        print('pivoit_value is ', pivoit_value)
        high -= 1
        
        while low + 1 < high:
            print(L[low:high])
            print('[', low, ',', high, ')')
            if L[low] < pivoit_value:
                low += 1
                low_toex = None
            else:
                low_toex = low
            if L[high] >= pivoit_value:
                high -= 1
                high_toex = None
            else:
                high_toex = high
                
            if low_toex and high_toex:
                self.swap(L, low_toex, high_toex)
                low_toex, high_toex = None, None
                
        return high

In [54]:
mq = QuickSort()
mq.sort([1,5,2,7,9,3])

==========================>>> partition
pivoit_value is  4.5
[1, 5, 2, 7, 9]
[ 0 , 5 )
[5, 2, 7, 9]
[ 1 , 5 )
[3, 2, 7, 9]
[ 1 , 5 )
[2, 7]
[ 2 , 4 )
==========================>>> partition
pivoit_value is  2.0
[1, 3]
[ 0 , 2 )
==========================>>> partition
pivoit_value is  2.5
==========================>>> partition
pivoit_value is  7.0
[7, 9]
[ 3 , 5 )
[5, 9]
[ 3 , 5 )
==========================>>> partition
pivoit_value is  8.0
[1, 5, 2, 7, 9, 3]
inversion number: 4
[1, 3, 2, 5, 9, 7] False
------------------------
N = 6
========= Theretical Analysis =========


NotImplementedError: 